## Parameters

In [1]:
library(Seurat)
library(ggplot2)

Attaching SeuratObject



In [12]:
library(Signac)

In [8]:
library(patchwork)


In [2]:
indir <- "/data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/Flt3l/outs/"
indir_mgatk <- "/data2/mito_lineage/data/processed/mttrace/CHIP_april08_2021_Croker/MTblacklist/Flt3l/MT/cellr_True/Flt3l_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/"
name <- "Flt3l"
outdir <- "/data2/mito_lineage/Analysis/annotation/output/data/"


In [3]:
anno_gtf <- "/data/isshamie/mito_lineage/data/external/GRCh38_MT_blacklist/genes/genes.gtf"

In [4]:
peaks_cd34_f <- "/data2/mito_lineage/Analysis/annotation/data/granja_cd34/GSE129785_scATAC-Hematopoiesis-CD34.peaks.bed"

## Gene activity quantification

In [5]:
peaks <- Read10X_h5(file.path(indir, "filtered_peak_bc_matrix.h5"))
# create a gene activity matrix from the peak matrix and GTF, using chromosomes 1:22, X, and Y.
# Peaks that fall within gene bodies, or 2kb upstream of a gene, are considered


Warning message in sparseMatrix(i = indices[] + 1, p = indptr[], x = as.numeric(x = counts[]), :
“'giveCsparse' has been deprecated; setting 'repr = "T"' for you”


In [10]:

# To install the current release
install.packages("Signac")

# To install the development version
install.packages("devtools")
devtools::install_github("timoast/signac", ref = "develop")

also installing the dependencies ‘SnowballC’, ‘tweenr’, ‘slam’, ‘sparsesvd’, ‘docopt’, ‘ggseqlogo’, ‘fastmatch’, ‘lsa’, ‘RcppRoll’, ‘ggforce’, ‘qlcMatrix’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom




Skipping 6 packages not available: BiocGenerics, S4Vectors, Rsamtools, IRanges, GenomicRanges, GenomeInfoDb



✔  checking for file ‘/tmp/RtmpEPZK0t/remotes4b6572716fa5/timoast-signac-5d1c5f4/DESCRIPTION’
─  preparing ‘Signac’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  installing the package to process help pages
─  saving partial Rd database (58.6s)
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
   Removed empty directory ‘Signac/.github’
   Removed empty directory ‘Signac/vignettes’
─  building ‘Signac_1.2.1.9002.tar.gz’
   


In [11]:
activity.matrix <- Signac::GeneActivity(peak.matrix = peaks, annotation.file = anno_gtf, 
    seq.levels = c(1:22, "X", "Y"), upstream = 2000, verbose = TRUE)

ERROR: Error in DefaultAssay(object = object): argument "object" is missing, with no default


## Object setup

In [ ]:
pbmc.atac <- CreateSeuratObject(counts = peaks, assay = "ATAC", project = "10x_ATAC")
pbmc.atac[["ACTIVITY"]] <- CreateAssayObject(counts = activity.matrix)
meta <- read.table(file.path(indir_mgatk,"vireoIn", "cellSNP.samples.tsv", sep = ",", header = TRUE, row.names = 1, 
    stringsAsFactors = FALSE)
meta <- meta[colnames(pbmc.atac), ]
pbmc.atac <- AddMetaData(pbmc.atac, metadata = meta)
pbmc.atac <- subset(pbmc.atac, subset = nCount_ATAC > 5000)
pbmc.atac$tech <- "atac"

## Data preprocessing

In [ ]:
DefaultAssay(pbmc.atac) <- "ACTIVITY"
pbmc.atac <- FindVariableFeatures(pbmc.atac)
pbmc.atac <- NormalizeData(pbmc.atac)
pbmc.atac <- ScaleData(pbmc.atac)

In [ ]:
DefaultAssay(pbmc.atac) <- "ATAC"
VariableFeatures(pbmc.atac) <- names(which(Matrix::rowSums(pbmc.atac) > 100))
pbmc.atac <- RunLSI(pbmc.atac, n = 50, scale.max = NULL)
pbmc.atac <- RunUMAP(pbmc.atac, reduction = "lsi", dims = 1:50)

## Plot

In [ ]:
p1 <- DimPlot(pbmc.atac, reduction = "umap") + NoLegend() + ggtitle("scATAC-seq")

## Incorporate RNA data

In [ ]:
pbmc.rna <- readRDS("../data/pbmc_10k_v3.rds")
pbmc.rna$tech <- "rna"

In [ ]:
p1 <- DimPlot(pbmc.atac, reduction = "umap") + NoLegend() + ggtitle("scATAC-seq")
p2 <- DimPlot(pbmc.rna, group.by = "celltype", label = TRUE, repel = TRUE) + NoLegend() + ggtitle("scRNA-seq")
CombinePlots(plots = list(p1, p2))

## Transfer labels from scRNA to scATAC

In [ ]:
transfer.anchors <- FindTransferAnchors(reference = pbmc.rna, query = pbmc.atac, features = VariableFeatures(object = pbmc.rna), 
    reference.assay = "RNA", query.assay = "ACTIVITY", reduction = "cca")

In [ ]:
celltype.predictions <- TransferData(anchorset = transfer.anchors, refdata = pbmc.rna$celltype, 
    weight.reduction = pbmc.atac[["lsi"]])
pbmc.atac <- AddMetaData(pbmc.atac, metadata = celltype.predictions)

In [ ]:
hist(pbmc.atac$prediction.score.max)
abline(v = 0.5, col = "red")

In [ ]:
table(pbmc.atac$prediction.score.max > 0.5)


In [ ]:
pbmc.atac.filtered <- subset(pbmc.atac, subset = prediction.score.max > 0.5)
pbmc.atac.filtered$predicted.id <- factor(pbmc.atac.filtered$predicted.id, levels = levels(pbmc.rna))  # to make the colors match
p1 <- DimPlot(pbmc.atac.filtered, group.by = "predicted.id", label = TRUE, repel = TRUE) + ggtitle("scATAC-seq cells") + 
    NoLegend() + scale_colour_hue(drop = FALSE)
p2 <- DimPlot(pbmc.rna, group.by = "celltype", label = TRUE, repel = TRUE) + ggtitle("scRNA-seq cells") + 
    NoLegend()
CombinePlots(plots = list(p1, p2))

In [ ]:
# note that we restrict the imputation to variable genes from scRNA-seq, but could impute the
# full transcriptome if we wanted to
genes.use <- VariableFeatures(pbmc.rna)
refdata <- GetAssayData(pbmc.rna, assay = "RNA", slot = "data")[genes.use, ]

# refdata (input) contains a scRNA-seq expression matrix for the scRNA-seq cells.  imputation
# (output) will contain an imputed scRNA-seq matrix for each of the ATAC cells
imputation <- TransferData(anchorset = transfer.anchors, refdata = refdata, weight.reduction = pbmc.atac[["lsi"]])

# this line adds the imputed data matrix to the pbmc.atac object
pbmc.atac[["RNA"]] <- imputation
coembed <- merge(x = pbmc.rna, y = pbmc.atac)

# Finally, we run PCA and UMAP on this combined object, to visualize the co-embedding of both
# datasets
coembed <- ScaleData(coembed, features = genes.use, do.scale = FALSE)
coembed <- RunPCA(coembed, features = genes.use, verbose = FALSE)
coembed <- RunUMAP(coembed, dims = 1:30)
coembed$celltype <- ifelse(!is.na(coembed$celltype), coembed$celltype, coembed$predicted.id)

In [ ]:
p1 <- DimPlot(coembed, group.by = "tech")
p2 <- DimPlot(coembed, group.by = "celltype", label = TRUE, repel = TRUE)
CombinePlots(list(p1, p2))